# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [60]:
# 1) Seu código aqui


# Definindo a URL da API do Tesouro Nacional para PVL

url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'

# Fazendo a requisição GET para obter os dados da API

r = requests.get(url)

# Convertendo a resposta da API em formato JSON

data = r.json()

# Criando um DataFrame do Pandas com os dados obtidos

df = pd.DataFrame(data['items'])





In [61]:

# Filtrando as linhas do DataFrame 'df' onde a coluna 'uf' tem o valor 'RJ' e criando uma cópia desse subconjunto em 'df_RJ_municipios'

df_RJ_municipios = df[df['uf'] == 'RJ'].copy()





In [64]:
# 2) Seu código aqui

# Contando a frequência de cada 'status' no DataFrame 'df_RJ_municipios' e pegando os três status mais frequentes

top_3_status = df_RJ_municipios['status'].value_counts().head(3)
print(top_3_status)

Deferido                                                 35
Encaminhado à PGFN com manifestação técnica favorável    13
Arquivado                                                11
Name: status, dtype: int64


In [74]:
# 3) Seu código aqui

# Criando uma nova coluna 'ano_status' no DataFrame 'df_RJ_municipios'
# Esta coluna contém apenas o ano, extraído da coluna 'data_status'
df_RJ_municipios['ano_status'] = df_RJ_municipios['data_status'].str.slice(6, 10)


In [75]:
# Contando a frequência de cada ano único na coluna 'ano_status' do DataFrame 'df_RJ_municipios'

frequencia_anos = df_RJ_municipios['ano_status'].value_counts()
print(frequencia_anos)

2007    11
2008     9
2014     8
2011     5
2020     5
2006     4
2002     4
2016     3
2023     3
2017     3
2012     3
2009     3
2013     3
2021     2
2010     2
2015     2
2018     2
2003     1
2019     1
2004     1
2022     1
Name: ano_status, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [45]:
#1) Seu código aqui

def consulta_API(uf, tipo_interessado):
    # Definindo a URL base da API do Tesouro Nacional

    base_url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    
    # Definindo os parâmetros da consulta com base nos argumentos fornecidos

    params = {"uf": uf, "tipo_interessado": tipo_interessado}
    
    # Fazendo a requisição GET à API

    response = requests.get(base_url, params=params)
    
    # Convertendo a resposta em formato JSON

    data = response.json()
    
    # Criando um DataFrame do Pandas com os dados retornados pela API

    df = pd.DataFrame(data['items'])
    
    
    return df



,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,7301,Município,João Monlevade,3136207,MG,None,Deferido,17944.000541/2010-61,2010-07-06T03:00:00Z,Operação contratual interna,RELUZ,Empresa Não Estatal,Cemig Distribuição S/A,Real,2.386582e+06,1,0,28/07/2010
1,7302,Município,Treze de Maio,4218400,SC,None,Deferido,17944.000541/2011-41,2011-05-16T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado de Santa Catarina...,Real,1.000000e+06,1,0,16/06/2011
2,7306,Município,Viana,3205101,ES,None,Deferido,17944.000542/2008-91,2008-05-23T03:00:00Z,Operação contratual interna,RELUZ,Empresa Não Estatal,Espírito Santo Centrais Elétricas S/A,Real,1.229982e+06,0,0,30/05/2008
3,7322,Município,Piracicaba,3538709,SP,None,Arquivado por decurso de prazo,17944.000545/2010-49,2011-08-11T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado de São Paulo,Real,2.253913e+06,0,0,17/01/2013
4,7329,Estado,Maranhão,21,MA,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000547/2009-02,2009-09-18T03:00:00Z,Operação contratual interna (com garantia da U...,PEF - Programa Emergencial de Financiamento do...,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,2.887280e+08,1,0,02/10/2009


In [76]:
# 2) Seu código aqui

# Consultando a API para obter dados das solicitações feitas pelo estado de Minas Gerais

df_MG = consulta_API("MG", "Estado")


# Filtrando o DataFrame para conter apenas as solicitações que foram arquivadas por decurso de prazo
# e contando o número de tais solicitações
solicitacoes_arquivadas = df_MG[df_MG['status'] == 'Arquivado por decurso de prazo'].shape[0]

print(solicitacoes_arquivadas)




1


In [77]:
# 3) Seu código aqui

# Consultando a API para obter dados das solicitações feitas pelos municípios do estado da Bahia

df_BA = consulta_API("BA", "Município")


# Filtrando as solicitações que foram deferidas, agrupando-as por município
# e contando o número de solicitações deferidas para cada município

municipios_deferidos = df_BA[df_BA['status'] == 'Deferido'].groupby('interessado').size()

# Identificando o município com o maior número de solicitações deferidas

municipio_mais_solicitacoes = municipios_deferidos.idxmax()


print(municipio_mais_solicitacoes)




Luís Eduardo Magalhães


In [59]:
# 4) Seu código aqui

# Consultando a API para obter dados das solicitações feitas pelo estado da Bahia

df_BA_estado = consulta_API("BA", "Estado")

# Salvando o DataFrame em um arquivo CSV chamado "solicitacoes_Bahia.csv"

df_BA_estado.to_csv("solicitacoes_Bahia.csv", index=False)
